<a href="https://colab.research.google.com/github/sbbwoy/ESAA/blob/main/220513_Doc_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import glob, os

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = '/content/drive/MyDrive/ESAA/OpinosisDataset1.0/OpinosisDataset1.0/topics'
#path 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트화
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

#개별 파일의 파일명은 filenames_list로 취합
#개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
    #개별 파일을 읽어 df로 생성
    df = pd.read_table(file_, index_col = None, header = 0, encoding = 'latin1')

    #절대 경로로 주어진 파일명을 가공, 확장자 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 list와 파일 내용 list에 파일명과 파일 내용 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

#파일명 list와 파일 내용 list 객체를 df로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,buttons_amazon_kindle,...
1,comfort_toyota_camry_2007,...
2,battery-life_ipod_nano_8gb,...
3,display_garmin_nuvi_255W_gps,...
4,battery-life_amazon_kindle,...


In [9]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

def LemTokens(tokens):
    lemmer = nltk.stem.WordNetLemmatizer()
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [10]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [16]:
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words = 'english',
                             ngram_range=(1, 2), min_df = 0.05, max_df = 0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


In [18]:
#5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters = 5, max_iter = 10000, random_state = 0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,buttons_amazon_kindle,...,2
1,comfort_toyota_camry_2007,...,1
2,battery-life_ipod_nano_8gb,...,3
3,display_garmin_nuvi_255W_gps,...,2
4,battery-life_amazon_kindle,...,3


In [22]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
16,food_holiday_inn_london,...,0
22,food_swissotel_chicago,...,0
18,free_bestwestern_hotel_sfo,...,0
12,location_bestwestern_hotel_sfo,...,0
17,location_holiday_inn_london,...,0
34,parking_bestwestern_hotel_sfo,...,0
33,price_amazon_kindle,...,0
27,price_holiday_inn_london,...,0
40,service_bestwestern_hotel_sfo,...,0
41,service_holiday_inn_london,...,0


In [23]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
7,comfort_honda_accord_2008,...,1
1,comfort_toyota_camry_2007,...,1
13,gas_mileage_toyota_camry_2007,...,1
20,interior_honda_accord_2008,...,1
15,interior_toyota_camry_2007,...,1
14,mileage_honda_accord_2008,...,1
25,performance_honda_accord_2008,...,1
24,quality_toyota_camry_2007,...,1
35,seats_honda_accord_2008,...,1
48,transmission_toyota_camry_2007,...,1


In [24]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
9,accuracy_garmin_nuvi_255W_gps,...,2
0,buttons_amazon_kindle,...,2
5,directions_garmin_nuvi_255W_gps,...,2
3,display_garmin_nuvi_255W_gps,...,2
6,eyesight-issues_amazon_kindle,...,2
19,features_windows7,...,2
11,fonts_amazon_kindle,...,2
21,keyboard_netbook_1005ha,...,2
30,navigation_amazon_kindle,...,2
29,satellite_garmin_nuvi_255W_gps,...,2


In [25]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
4,battery-life_amazon_kindle,...,3
2,battery-life_ipod_nano_8gb,...,3
10,battery-life_netbook_1005ha,...,3
32,performance_netbook_1005ha,...,3
44,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,3


In [26]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
8,bathroom_bestwestern_hotel_sfo,...,4
23,room_holiday_inn_london,...,4
28,rooms_bestwestern_hotel_sfo,...,4
26,rooms_swissotel_chicago,...,4


In [27]:
#3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter = 10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [28]:
#소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
25,performance_honda_accord_2008,...,0
24,quality_toyota_camry_2007,...,0
15,interior_toyota_camry_2007,...,0
14,mileage_honda_accord_2008,...,0
13,gas_mileage_toyota_camry_2007,...,0
35,seats_honda_accord_2008,...,0
7,comfort_honda_accord_2008,...,0
20,interior_honda_accord_2008,...,0
48,transmission_toyota_camry_2007,...,0
1,comfort_toyota_camry_2007,...,0


In [32]:
document_df[document_df['cluster_label']==0].sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
1,comfort_toyota_camry_2007,...,0
7,comfort_honda_accord_2008,...,0
13,gas_mileage_toyota_camry_2007,...,0
14,mileage_honda_accord_2008,...,0
15,interior_toyota_camry_2007,...,0
20,interior_honda_accord_2008,...,0
24,quality_toyota_camry_2007,...,0
25,performance_honda_accord_2008,...,0
35,seats_honda_accord_2008,...,0
48,transmission_toyota_camry_2007,...,0


In [34]:
document_df[document_df['cluster_label']==1].sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
8,bathroom_bestwestern_hotel_sfo,...,1
12,location_bestwestern_hotel_sfo,...,1
16,food_holiday_inn_london,...,1
17,location_holiday_inn_london,...,1
18,free_bestwestern_hotel_sfo,...,1
22,food_swissotel_chicago,...,1
23,room_holiday_inn_london,...,1
26,rooms_swissotel_chicago,...,1
27,price_holiday_inn_london,...,1
28,rooms_bestwestern_hotel_sfo,...,1


In [35]:
document_df[document_df['cluster_label']==2].sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,buttons_amazon_kindle,...,2
46,video_ipod_nano_8gb,...,2
45,size_asus_netbook_1005ha,...,2
44,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,2
42,screen_ipod_nano_8gb,...,2
39,speed_garmin_nuvi_255W_gps,...,2
38,speed_windows7,...,2
36,screen_netbook_1005ha,...,2
33,price_amazon_kindle,...,2
32,performance_netbook_1005ha,...,2


**군집별 핵심 단어 추출하기**

KMeans 객체의 clusters_centers_: 각 군집을 구성하는 단어 피처가 군집을 중심을 기준으로 얼마나 가까이 위치해 있는지 수치화. 1에 가까울수록 중심과 가까움을 의미

In [36]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [42]:
#군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    #cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
    #군집 중심점별 할당된 word 피처들의 거리값이 큰 순으로 값을 바꾸기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        #개별 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        #cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]

        #top_feature_indexes를 이요해 해당 피처 단어의 중심 위치 상대값 계산
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        #cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] ==cluster_num]['filename']
        filenames = filenames.values.tolist()

        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [38]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('##### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명:', cluster_detail['filenames'][:7])
        print('================================================')

In [43]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num = 3, top_n_features = 10)
print_cluster_details(cluster_details)

##### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_toyota_camry_2007', 'comfort_honda_accord_2008', 'gas_mileage_toyota_camry_2007', 'mileage_honda_accord_2008', 'interior_toyota_camry_2007', 'interior_honda_accord_2008', 'quality_toyota_camry_2007']
##### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'location_holiday_inn_london', 'free_bestwestern_hotel_sfo', 'food_swissotel_chicago', 'room_holiday_inn_london']
##### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명: ['buttons_amazon_kindle', 'battery-life_ipod_nano_8gb', 'display_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'directio

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
